In [7]:
import os
import spotipy as sp
import requests
import pandas as pd
import numpy as np
import lyricsgenius as lg
import re
import altair as alt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sb
import requests, json
import streamlit as st
import ipywidgets as widgets
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from datetime import datetime, timedelta
from IPython.display import Image, display, clear_output
from ipywidgets import Layout

pd.options.display.max_columns = 200
%matplotlib inline

Client_ID = "b12b8a17114f406abd4fffb4d0f3f760"
Client_Secret = "e2c2aef22ea344e9acf9d0af785d536c"

manager = SpotifyClientCredentials(
    Client_ID ,
    Client_Secret
)

geniusClient = "Qn169Xw5H5kxvBJAcvByD2ozld2IwEq5TtKQsH3JV9be-R7ION3J-kXGa4JolQgR"
geniusSecret = "4GToSYfHujKrk462TjRrGh1cLYgnN7GSR7QinC08AXrjAtgJyQLtHyRhGcSTmNdJljXls1az74RZ53KB-hFzKg"
geniusClientAccess = "cocQW79g3DkXey_cBNEASSuRzdKGA24Rmp4KW47MFNW6t4RaknXIA9EnI-PfxhoC"

sp = sp.Spotify(client_credentials_manager = manager)
genius = lg.Genius(geniusClientAccess)

In [8]:
def get_artist_info(artist_name):
    results = sp.search(q = 'artist:' + artist_name, type = 'artist', limit = 1)
    items = results['artists']['items']
    if len(items) > 0:
        artist = items[0]
        photo_url = artist['images'][-1]['url'] if artist['images'] else 'No image available'
    
        # Retrieve top tracks separately
        top_tracks_info = sp.artist_top_tracks(artist['id'], country = 'US')['tracks']
        top_tracks_list = [
            {
                'Rank': i + 1,
                'Name': track['name'],
                'Popularity': track['popularity'],
                'ID':track['id']
            } for i, track in enumerate(top_tracks_info)
        ]
        
        artist_albums = sp.artist_albums(artist['id'], limit = 5)
        artist_list = [
            {
                'Rank': i + 1,
                'Name': album['name'],
                'Date': album['release_date']
            } for i, album in enumerate(artist_albums['items']) 
        ]
        
        related_artists_response = sp.artist_related_artists(artist['id'])
        related_artists = related_artists_response['artists']  
        related_list = [
            {
                'Rank' : i + 1,
                'Name' : related_artist['name'],
                'Genres' : ''.join(related_artist['genres'][:1]),
                'Popularity' : related_artist['popularity']
            } for i, related_artist in enumerate(related_artists)
        ]
        
        
        
        artist_info = {
            'ID': artist['id'],
            'Name': artist['name'],
            'Genres': artist['genres'][:3],
            'Popularity': artist['popularity'],
            'Followers': artist['followers']['total'],
            'Spotify URL': artist['external_urls']['spotify'],
            'Photo': photo_url,
            'Top Tracks': top_tracks_list,
            'Artist Albums': artist_list,
            'Related Artists': related_list
        }
        return artist_info
    else:
        return "Artist not found."


In [9]:
spotify_user_id_input = widgets.Text(
    description = 'Spotify User ID:',
    placeholder = 'Enter your Spotify User ID',
    style = {'description_width': 'initial'}  # Adjusts the width of the label to prevent it from being cut off
)

song_info_output = widgets.Output()

def create_track_button(track_info):
    track_button = widgets.Button(description = track_info['Name'],
                                 layout = Layout(width = '300px'))
    track_id = track_info['ID'] 
    
    def on_track_button_clicked(b):
        with song_info_output:
            song_info_output.clear_output(wait = True)
            audio_analysis = sp.audio_features(track_id)  
            
            songsList = []
            
            track = sp.track(track_id)
            artist_name = track['artists'][0]['name']
            
            danceability = 0 # if there aren't audio features we get an error without these
            energy = 0 # creating theses allows us to still add the song to the songsList
            key = 0
            loudness = 0
            tempo = 0
            speechiness = 0
            acousticness = 0
            instrumentalness = 0
            liveness = 0
            valence = 0
            features = sp.audio_features([track_id])[0] # Essential for getting the first audio feature and not all
                
            if audio_analysis:
                danceability = features['danceability']
                energy = features['energy']
                key = features['key']
                loudness = features['loudness']
                tempo = features['tempo']
                speechiness = features['speechiness']
                acousticness = features['acousticness']
                instrumentalness = features['instrumentalness']
                liveness = features['liveness']
                valence = features['valence']
                print(f"Danceability: {danceability}") # Create figma board that explains these
                print(f"Energy: {energy}")
                print(f"Key: {key}")
                print(f"Loudness: {loudness}")
                print(f"Tempo: {tempo}")
                print(f"Speechiness: {speechiness}") # add explanation of what each of these means (FIGMA)
                print(f"Acousticness: {acousticness}")
                print(f"Instrumentalness: {instrumentalness}")
                print(f"Liveness: {liveness}")
                print(f"Valence: {valence}")
            else:
                print("No Audio Features Available!") # some songs that are new to Spotify will not have features
                
            print("")
            
            artistID = get_artist_info
            
            geniusSong = genius.search_song(track_info["Name"], artist_name) #artistID
                
            if geniusSong is not None: 
                lyrics = geniusSong.lyrics # Class 2-8 Notes, cleanCruftyLyrics function
                cleanedLyrics = re.sub("\d+ Contributor.+Lyrics", "", lyrics)
                cleanedLyrics = re.sub("[\!\.\?\'\$\w ]+You might also like\n", "", cleanedLyrics)
                cleanedLyrics = re.sub("\d*Embed$", "", cleanedLyrics)
                if cleanedLyrics.count("You might also like\n") == 1:
                    cleanedLyrics = cleanedLyrics.replace("You might also like\n", "")
                cleanedLyrics = re.sub("You might also like$", "", cleanedLyrics)
                cleanedLyrics = re.sub(r'\[.*?\]', '', cleanedLyrics) # https://docs.python.org/3/library/re.html

            tokenizedLyrics = []
            wordList = cleanedLyrics.split()
        
            for words in wordList: # tokenizing the words and putting them in our list (getting tokens)
                tokens = regexp_tokenize(words, "[\w']+")
                tokenizedLyrics.extend(tokens) # .extend creates a list of strings where
                                               # .append made a list of lists (W3 Schools)
        
            stopWords = set(stopwords.words('english')) # find the stop words in the lyrics
            allStopWords = [] # gets rid of words like "I", "am", "the", etc.
            stopWordsList = []
            for word in tokenizedLyrics:
                if word.lower() not in stopWords:
                    stopWordsList.append(word.lower())
            allStopWords.extend(stopWordsList)

            wordCount = {}
            for word in allStopWords:
                if word in wordCount:
                    wordCount[word] += 1
                else:
                    wordCount[word] = 1
                
            print("")
            print("Sentiment Analysis:")
                
            print("") 

            vaderSIA = SentimentIntensityAnalyzer()
            vaderScore = vaderSIA.polarity_scores(cleanedLyrics)
        
            print(vaderScore)
            print("")
    
            trackInfo = {"Song": track_id, "Artist": artist_name, "ID": track_id,
                         "Danceability": danceability, "Energy": energy, "Key": key, "Loudness": loudness,
                         "Tempo": tempo, "Speechiness": speechiness, "Acousticness": acousticness,
                         "Instrumentalness": instrumentalness, "Liveness": liveness, "Valence": valence,
                         "Lyrics": cleanedLyrics, "Stop Words": allStopWords, "Word Count": wordCount,
                        "Vader Score": vaderScore}
            songsList.append(trackInfo) # dictionary that keeps all of the information that we have collected

            fullWordCount = {}
            for song in songsList: # Counting all the words just this time in songList (every song)
                wordCount = song["Word Count"]
                for word, count in wordCount.items(): # W3 Schools
                    lowercaseWord = word.lower() # lowercase each word for an accurate number
                    if lowercaseWord in fullWordCount:
                        fullWordCount[lowercaseWord] += count
                    else:
                        fullWordCount[lowercaseWord] = count
    
            totalSA = {'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0} # create dict and set all values to 0
            for song in songsList: # loop through each songs vader score 
                totalVader = song["Vader Score"]
                totalSA['neg'] += totalVader['neg']
                totalSA['neu'] += totalVader['neu']
                totalSA['pos'] += totalVader['pos']
                totalSA['compound'] += totalVader['compound']
    
            fullSA = {'neg': totalSA['neg'] / 10, # divide by 10 to get the average score for each category
                      'neu': totalSA['neu'] / 10,
                      'pos': totalSA['pos'] / 10,
                      'compound': totalSA['compound'] / 10 }
                  
            word_counts_df = pd.DataFrame(list(wordCount.items()), columns=['Word', 'Count'])

            # Assuming the word count DataFrame `word_counts_df` is correctly created as before
            word_count_chart = alt.Chart(word_counts_df.nlargest(10, 'Count')).mark_bar().encode(
                x=alt.X('Word:N', sort='-y', title='Word'),
                y=alt.Y('Count:Q', title='Count'),
                color='Word:N'
            ).properties(
                title='Top 10 Most Frequent Words',
                width=400,
                height=300
            )

            # Display the charts side by side
            charts = word_count_chart
            charts.display()
            
            
    track_button.on_click(on_track_button_clicked)
    return track_button


def create_and_populate_playlist(artist_name, user_id):
    user_id = spotify_user_id_input.value
    if not user_id:
        print("Please enter your Spotify User ID")
    artist_result = sp.search(q = "artist:" + artist_name, type = "artist", limit = 1)
    if artist_result['artists']['items']:
        original_artist_id = artist_result['artists']['items'][0]['id']
    else:
        print("Artist not found.")
        return
    
    related_artists = sp.artist_related_artists(original_artist_id)
    related_artist_ids = [artist['id'] for artist in related_artists['artists']][:5]  # Limit to 5 related artists

    track_uris = []
    for artist_id in [original_artist_id] + related_artist_ids:
        top_tracks = sp.artist_top_tracks(artist_id, country='US')['tracks']
        track_uris.extend([track['uri'] for track in top_tracks][:5])  # Limit to 5 top tracks per artist

    playlist_name = f"Playlist Based on {artist_name}"
    new_playlist = sp.user_playlist_create(user = user_id, name = playlist_name, public = True)

    sp.user_playlist_add_tracks(user = user_id, playlist_id = new_playlist['id'], tracks = track_uris)

    print(f"Playlist created: {new_playlist['external_urls']['spotify']}")

def on_create_playlist_button_clicked(b):
    user_id = spotify_user_id_input.value
    artist_name = artist_name_input.value
    if user_id and artist_name:
        create_and_populate_playlist(artist_name)  # Function to create the playlist
    else:
        print("Please make sure the artist name and your Spotify user ID are provided.")


def on_search_button_clicked(b):
    with output:
        clear_output(wait = True)
        artist_name = artist_name_input.value
        if artist_name:
            artist_info = get_artist_info(artist_name)
            if isinstance(artist_info, dict):
                tab = widgets.Tab()
                tabs_titles = ['Artist Info', 'Top Tracks', 'Albums', 'Artists You May Like','Create a playlist']
                tab.children = [widgets.VBox(), widgets.VBox(), widgets.VBox(), widgets.VBox(), widgets.VBox()]  # Create a VBox for each tab
                
                # Artist Info Tab
                info_content = widgets.HTML(
                    value = f"""
                    <h2>{artist_info['Name']}</h2>
                    <img src="{artist_info['Photo']}" alt="Artist photo" style="max-height: 200px;"><br>
                    <b>Genres:</b> {', '.join(artist_info['Genres'])}<br>
                    <b>Popularity:</b> {artist_info['Popularity']}<br>
                    <b>Followers:</b> {artist_info['Followers']:,}<br>
                    <a href="{artist_info['Spotify URL']}" target="_blank">Spotify Profile</a>
                    """
                )
                tab.children[0].children = [info_content]
                
                # Top Tracks Tab
                top_tracks_buttons = [create_track_button(track) for track in artist_info['Top Tracks']]
                tracks_content = widgets.VBox(children = top_tracks_buttons + [song_info_output])  # Include the song_info_output widget
                tab.children[1].children = [tracks_content]
                
                # Albums Tab
                albums_content = widgets.VBox([
                    widgets.HTML(value = f"<b>{i}. {album['Name']}</b> (Released: {album['Date']})")
                    for i, album in enumerate(artist_info['Artist Albums'], start = 1)
                ])
                tab.children[2].children = [albums_content]
                
                # Related Artists Tab
                related_content = widgets.VBox([
                    widgets.HTML(value = f"<b>{i}. {artist['Name']}</b> (Genres: {''.join(artist['Genres'])})")
                    for i, artist in enumerate(artist_info['Related Artists'], start=1)
                ])
                tab.children[3].children = [related_content]
                
                spotify_user_id_input = widgets.Text(
                    description = 'Spotify User ID:',
                    placeholder = 'Enter your Spotify User ID',
                    style = {'description_width': 'initial'}  
                )
                # Playlist tab
                create_playlist_button = widgets.Button(description = "Create Playlist")
                create_playlist_button.on_click(on_create_playlist_button_clicked)  
                
                tab.children[4].children = [spotify_user_id_input, create_playlist_button] 
                
                for i, title in enumerate(tabs_titles):
                    tab.set_title(i, title)
                
                display(tab)
            else:
                print("Artist not found or an error occurred")
                

artist_name_input = widgets.Text(description = 'Artist Name:')
search_button = widgets.Button(description = "Search")
output = widgets.Output()
song_info_output = widgets.Output()

search_button.on_click(on_search_button_clicked)

display(artist_name_input, search_button, output)

Text(value='', description='Artist Name:')

Button(description='Search', style=ButtonStyle())

Output()